## LCEL Tinkering

> https://python.langchain.com/docs/expression_language/get_started

---

In [41]:
from dotenv import load_dotenv
from operator import itemgetter
import os

In [42]:
load_dotenv("/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/knowledge_graphs/.env")

True

In [56]:
from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string

from langchain_community.vectorstores import DocArrayInMemorySearch

In [44]:
chat_gpt4 = AzureChatOpenAI(
        deployment_name="gpt-4-32k",
        model="gpt-4-32k",
        openai_api_type="azure",
        azure_endpoint=os.environ['AZURE_API_BASE'],
        openai_api_key=os.environ['AZURE_API_KEY'],
        openai_api_version=os.environ['AZURE_API_VERSION'],
        max_retries=2,
        temperature=0,
    )
embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_type='azure',
    azure_endpoint=os.environ['AZURE_API_BASE'],
    openai_api_key=os.environ['AZURE_API_KEY'],
    openai_api_version=os.environ['AZURE_API_VERSION'],
    max_retries=5
)

In [45]:
gpt_35_turbo = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    model="gpt-35-turbo",
    openai_api_type="azure",
    azure_endpoint=os.environ['AZURE_API_BASE'],
    openai_api_key=os.environ['AZURE_API_KEY'],
    openai_api_version=os.environ['AZURE_API_VERSION'],
    max_retries=2,
    temperature=1,
)

In [46]:
gpt_35_turbo_instruct = AzureOpenAI(
    deployment_name="gpt-35-turbo-instruct",
    model="gpt-35-turbo-instruct",
    openai_api_type="azure",
    azure_endpoint=os.environ['AZURE_API_BASE'],
    openai_api_key=os.environ['AZURE_API_KEY'],
    openai_api_version=os.environ['AZURE_API_VERSION'],
    max_retries=2,
    temperature=1,
)

In [47]:
chat_gpt4.invoke("hey")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}})

In [48]:
embedding.embed_query("hello")[:5]

[-0.0251408640862754,
 -0.019467308584339062,
 -0.028037156142099832,
 -0.031026021659306558,
 -0.024717661806730858]

## Basic example: prompt + model + output parser

In [14]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = gpt_35_turbo
output_parser = StrOutputParser()

chain = prompt | model | output_parser

user_input = {"topic": "cricket"}

chain.invoke(user_input)

'Why did the cricket team go on a picnic? \n\nBecause they wanted to take a good wicket!'

The `|` symbol is similar to a unix pipe operator, which chains together the different components feeds the output from one component as input into the next component.

In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model, then the model output is passed to the output parser. Let’s take a look at each component individually to really understand what’s going on.

### 1. prompt

prompt is a `BasePromptTemplate`, which means it takes in a dictionary of template variables and produces a PromptValue. A PromptValue is a wrapper around a completed prompt that can be passed to either an LLM (which takes a string as input) or ChatModel (which takes a sequence of messages as input). It can work with either language model type because it defines logic both for producing BaseMessages and for producing a string.

In [15]:
prompt_output = prompt.invoke(user_input)
prompt_output

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about cricket')])

### 2. Model

The PromptValue is then passed to model. In this case our model is a ChatModel, meaning it will output a BaseMessage.

In [16]:
model_output = model.invoke(prompt_output)

model_output

AIMessage(content='Why did the cricket team go to the bank?\nTo get a loan for a new bat because they were stumped!', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}})

In [17]:
type(model_output)

langchain_core.messages.ai.AIMessage

If our model was an LLM, it would output a string.


In [18]:
llm = gpt_35_turbo_instruct

llm.invoke(prompt_output)

'\n\nWhy did the cricket player go to the doctor? Because he was feeling a little bit crickety. '

### 3. Output parser



And lastly we pass our model output to the output_parser, which is a BaseOutputParser meaning it takes either a string or a BaseMessage as input. 
The StrOutputParser specifically simple converts any input into a string.

In [12]:
output_parser.invoke(model_output)

'Why did the cricket team go to the library? \n\nTo improve their bowling average!'

### 4. Entire Pipeline

To follow the steps along:

- We pass in user input on the desired topic as {"topic": "ice cream"}
- The prompt component takes the user input, which is then used to construct a PromptValue after using the topic to construct the prompt.
- The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a ChatMessage object.
- Finally, the output_parser component takes in a ChatMessage, and transforms this into a Python string, which is returned from the invoke method.

## RAG Search Example

In [19]:
texts = [
    "mini is 28 years old",
    "mini likes to eat paneer",
    "mini lives in kolkata, jhansi and bangalore"
]

In [20]:
vectorstore = DocArrayInMemorySearch.from_texts(
    texts, embedding=embedding
)

In [21]:
retriever = vectorstore.as_retriever()

In [23]:
retriever.invoke("mini like to eat what")

[Document(page_content='mini likes to eat paneer'),
 Document(page_content='mini is 28 years old'),
 Document(page_content='mini lives in kolkata, jhansi and bangalore')]

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [19]:
prompt = ChatPromptTemplate.from_template(template)
model = gpt_35_turbo
output_parser = StrOutputParser()

In [18]:
prompt.invoke({
    "context": "context",
    "question": "question"
})

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\ncontext\n\nQuestion: question\n')])

In [20]:
setup_and_retrieval = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
)

In [21]:
chain = setup_and_retrieval | prompt | model | output_parser

In [22]:
chain.invoke("where does mini live?")

'Mini lives in Kolkata, Jhansi and Bangalore.'

retriever takes as input:
- question
computes:
- context

prompt takes as input:

- question
- context

passes this to llm

llm takes as input:

- prompt

passes the llm response to a parser


![](https://i.imgur.com/i5nrGEB.jpg)

In this case, the composed chain is:

`chain = setup_and_retrieval | prompt | model | output_parser`

To explain this, we first can see that the prompt template above takes in context and question as values to be substituted in the prompt. Before building the prompt template, we want to retrieve relevant documents to the search and include them as part of the context.

As a preliminary step, we’ve setup the retriever using an in memory store, which can retrieve documents based on a query. This is a runnable component as well that can be chained together with other components, but you can also try to run it separately:

In [23]:
retriever.invoke("where does mini live?")

[Document(page_content='mini is 28 years old'),
 Document(page_content='mini lives in kolkata, jhansi and bangalore'),
 Document(page_content='mini likes to eat paneer')]

We then use the `RunnableParallel` to prepare the expected inputs into the prompt by using the entries for the retrieved documents as well as the original user question, using the retriever for document search, and RunnablePassthrough to pass the user’s question:

```
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
```

To review, the complete chain is:

```
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser
```


With the flow being:

1. The first steps create a `RunnableParallel` object with two entries. The first entry, `contex`t will include the document results fetched by the `retriever`. The second entry, `question` will contain the user’s original question. To pass on the question, we use `RunnablePassthrough` to copy this entry.

2. Feed the dictionary from the step above to the prompt component. It then takes the user input which is question as well as the retrieved document which is context to construct a prompt and output a `PromptValue`.

3. The `model` component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a `ChatMessage` object.

4. Finally, the output_parser component takes in a ChatMessage, and transforms this into a Python string, which is returned from the invoke method.


> As the prompt in the 2nd setep needs 2 outputs, we use a `RunnableParallel` in the `setup_and_retrieval` so that these 2 o/ps can be fed in parallel to the next component (prompt)

__What is `Runnable`__?


A unit of work that can be invoked, batched, streamed, transformed and composed.

The Runnable protocol is implemented for most components. This is a standard interface, which makes it easy to define custom chains as well as invoke them in a standard way. The standard interface includes:

- invoke/ainvoke: Transforms a single input into an output.

- batch/abatch: Efficiently transforms multiple inputs into outputs.

- stream/astream: Streams output from a single input as it’s produced.

- astream_log: Streams output and selected intermediate results from an input.

The input type and output type varies by component:

![](https://gcdnb.pbrd.co/images/LRnTQtlaNk9W.png)


All runnables expose input and output schemas to inspect the inputs and outputs: - input_schema: an input Pydantic model auto-generated from the structure of the Runnable - output_schema: an output Pydantic model auto-generated from the structure of the Runnable

Let’s take a look at these methods. To do so, we’ll create a super simple PromptTemplate + ChatModel chain.

In [24]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = gpt_35_turbo
output_parser = StrOutputParser()

chain = prompt | model | output_parser

user_input = {"topic": "cricket"}

chain.invoke(user_input)

'Why did the cricket player go to jail?\n\nBecause he bowled a maiden over!'

### Input Schema

In [25]:
# The input schema of the chain is the input schema of its first part, the prompt.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [26]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [28]:
model.input_schema.schema()

{'title': 'AzureChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': '

### Output Schema

A description of the outputs produced by a Runnable. This is a Pydantic model dynamically generated from the structure of any Runnable. You can call .schema() on it to obtain a JSONSchema representation.

In [29]:
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

In [30]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])


['Why did the teddy bear say no to dessert? \nBecause it was already stuffed!',
 "Why don't cats play poker in the jungle? Too many cheetahs!"]

In [31]:
# You can set the number of concurrent requests by using the max_concurrency parameter

chain.batch([{"topic": "bears"}, {"topic": "cats"}], config={"max_concurrency": 5})

["Why do bears have fur coats? \n\nBecause they'd look silly in leather!",
 'Why did the cat wear a fancy dress? Because she was feline fabulous!']

### Parallelism

Let’s take a look at how LangChain Expression Language supports parallel requests. For example, when using a RunnableParallel (often written as a dictionary) it executes each element in parallel.



In [32]:
chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [33]:
type(combined)

langchain_core.runnables.base.RunnableParallel

In [34]:
%%time
chain1.invoke({"topic": "bears"})

CPU times: user 9.7 ms, sys: 2.61 ms, total: 12.3 ms
Wall time: 1.36 s


AIMessage(content='Why did the bear break up with his girlfriend? \n\nBecause he found someone "furrier"!', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}})

In [35]:
%%time
chain2.invoke({"topic": "bears"})

CPU times: user 7.97 ms, sys: 1.78 ms, total: 9.75 ms
Wall time: 1.21 s


AIMessage(content='Big and brown, they roam around\nFierce and strong, they rule the ground.', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}})

In [36]:
%%time
combined.invoke({"topic": "bears"})

CPU times: user 22.2 ms, sys: 5.53 ms, total: 27.7 ms
Wall time: 1.39 s


{'joke': AIMessage(content='Why did the bear wear a raincoat?\n\nBecause it was just a drizzly bear!', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}),
 'poem': AIMessage(content='In the wild they roam,\nFurry giants, kings of home.', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}})}

In [38]:
%%time
combined.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: user 47.2 ms, sys: 8.18 ms, total: 55.4 ms
Wall time: 1.39 s


[{'joke': AIMessage(content="Why don't bears like fast food? Because they can't catch it!", response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}),
  'poem': AIMessage(content='Bears roam free, \nMajestic creatures of the wild.', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}})},
 {'joke': AIMessage(content='Why did the cat sit on the computer?\n\nTo keep an eye on the mouse!', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}),
  'poem': AIMessage(content="Soft, sleek, and sweet,\nA feline's purr can't be beat.", response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}})}]

## RAG with LCEL

> https://python.langchain.com/docs/expression_language/cookbook/retrieval

---

In [24]:
texts = [
    "mini is 28 years old",
    "mini likes to eat paneer",
    "mini lives in kolkata, jhansi and bangalore"
]

vectorstore = DocArrayInMemorySearch.from_texts(
    texts=texts, embedding=embedding
)

retriever = vectorstore.as_retriever()

In [25]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [26]:
prompt.invoke({"question": "q", "context": "c"})

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\nc\n\nQuestion: q\n')])

In [27]:
retriever.invoke("hello")

[Document(page_content='mini lives in kolkata, jhansi and bangalore'),
 Document(page_content='mini likes to eat paneer'),
 Document(page_content='mini is 28 years old')]

In [28]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [29]:
chain.invoke("what is mini's favorite food?")

"Mini's favorite food is paneer."

In [30]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

In [33]:
prompt.invoke({"context": "mini likes paneer", "question": "what is mini's favorite food?", "language": "italian"})

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\nmini likes paneer\n\nQuestion: what is mini's favorite food?\n\nAnswer in the following language: italian\n")])

- prompt expects 3 inputs:
    - context
    - question 
    - language

In [ ]:
# will throw an error - we only need to pass in question
retriever.invoke({"question": "what is mini's favorite food?", "language": "italian"})

In [34]:
chain_input = {"question": "what is mini's favorite food?", "language": "italian"}

In [35]:
retriever.invoke(chain_input['question'])

[Document(page_content='mini likes to eat paneer'),
 Document(page_content='mini is 28 years old'),
 Document(page_content='mini lives in kolkata, jhansi and bangalore')]

In [224]:
chain = (
    RunnableParallel({
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question") | RunnablePassthrough(),
        "language": itemgetter("language") | RunnablePassthrough(),
    })
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# ALSO WORKS 
# without RunnableParallel

# chain = (
#     {
#         "context": itemgetter("question") | retriever,
#         "question": itemgetter("question") | RunnablePassthrough(),
#         "language": itemgetter("language") | RunnablePassthrough(),
#     }
#     | prompt
#     | model
#     | StrOutputParser()
# )

In [39]:
chain.invoke({"question": "what does mini like to eat?", "language": "hindi"})

'मिनी को पनीर खाना पसंद है।'

## Multiple chains

> https://python.langchain.com/docs/expression_language/cookbook/multiple_chains

---

In [100]:
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

In [101]:
chain_input = {"person": "sachin tendulkar", "language": "bengali"}

In [102]:
chain1 = {"person": itemgetter("person")} | prompt1 | model | StrOutputParser()

In [103]:
chain1.invoke(chain_input)

'Sachin Tendulkar is from Mumbai, Maharashtra, India.'

In [104]:
chain2 = (
    {
    "city": chain1,
    "language": itemgetter("language") | RunnablePassthrough()
    }
    | prompt2
    | model
    | StrOutputParser()
)

In [105]:
chain2.invoke({"person": "sachin tendulkar", "language": "english"})

'The country that the city of Mumbai, India, where Sachin Tendulkar is from, is located in is India.'

### Chain.batch vs Chain.invoke

In [120]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [128]:
node_conversion_few_shot_examples = [
    {
        "city": """
        Kolkata
        """,
        "country": """
        India
        """
    }
]


In [129]:
node_converter_prefix = """
    You are an agent with expert knowledge in converting cities to country names
    """

In [130]:
example_template = """
    User: City:
    {city}
    AI: {country}
    """

In [133]:
# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["city", "country"],
    template=example_template
)

suffix = """
User: City:
{city}
AI:
"""

# now create the few-shot prompt template
few_shot_prompt_template_node_conversion = FewShotPromptTemplate(
    examples=node_conversion_few_shot_examples,
    example_prompt=example_prompt,
    prefix=node_converter_prefix,
    suffix=suffix,
    input_variables=["city"],
    example_separator="\n\n"
)

In [142]:
city_names = [
    "Tokyo",
    "New York",
    "London",
    "Paris",
    "Los Angeles",
    "Sydney",
    "Berlin",
    "Moscow",
    "Toronto",
    "Rio de Janeiro",
    "Mumbai",
    "Cairo",
    "Dubai",
    "Seoul",
    "Rome",
    "Bangkok",
    "Mexico City",
    "Amsterdam",
    "Singapore",
    "Istanbul"
]


In [143]:
chain = few_shot_prompt_template_node_conversion | model | StrOutputParser()

In [144]:
few_shot_prompt_template_node_conversion.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'city': {'title': 'City', 'type': 'string'}}}

In [145]:
input_mapping = [
    {"city": city_} for city_ in city_names
]
input_mapping

[{'city': 'Tokyo'},
 {'city': 'New York'},
 {'city': 'London'},
 {'city': 'Paris'},
 {'city': 'Los Angeles'},
 {'city': 'Sydney'},
 {'city': 'Berlin'},
 {'city': 'Moscow'},
 {'city': 'Toronto'},
 {'city': 'Rio de Janeiro'},
 {'city': 'Mumbai'},
 {'city': 'Cairo'},
 {'city': 'Dubai'},
 {'city': 'Seoul'},
 {'city': 'Rome'},
 {'city': 'Bangkok'},
 {'city': 'Mexico City'},
 {'city': 'Amsterdam'},
 {'city': 'Singapore'},
 {'city': 'Istanbul'}]

In [148]:
%%time
chain.batch(input_mapping)

CPU times: user 139 ms, sys: 14.2 ms, total: 153 ms
Wall time: 1.5 s


['Japan',
 'USA',
 'United Kingdom',
 'France',
 'United States of America',
 'Australia',
 'Germany',
 'Russia',
 'Canada',
 'Brazil',
 'India',
 'Egypt',
 'United Arab Emirates',
 'South Korea',
 'Italy',
 'Thailand',
 'Mexico',
 'Netherlands',
 'Singapore',
 'Turkey']

In [149]:
%%time
for city_ in city_names:
    print (chain.invoke({"city": city_}))

Japan
United States of America (USA)
United Kingdom
France
USA
Australia
Germany
Russia
Canada
Brazil
India
Egypt
United Arab Emirates
South Korea
Italy
Thailand
Mexico
Netherlands
Singapore
Turkey
CPU times: user 200 ms, sys: 12.6 ms, total: 213 ms
Wall time: 6.4 s


## Dynamically route logic based on input

> https://python.langchain.com/docs/expression_language/how_to/routing/

---

In [129]:
prompt = ChatPromptTemplate.from_template(
"""
Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:
"""
)

In [130]:
prompt.invoke({"question": "hello"})

ChatPromptValue(messages=[HumanMessage(content='\nGiven the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.\n\nDo not respond with more than one word.\n\n<question>\nhello\n</question>\n\nClassification:\n')])

In [131]:
chain = prompt | llm | StrOutputParser()

In [132]:
chain.invoke({"question": "how do I call Anthropic?"})

'\nAnthropic'

In [138]:
langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm | StrOutputParser()
anthropic_chain = PromptTemplate.from_template(
    """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm | StrOutputParser()
general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | llm | StrOutputParser()

You can also use a custom function to route between different outputs. Here’s an example:



In [147]:
def route(info):
    print ("Route info:", info)
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [144]:
from langchain_core.runnables import RunnableLambda

In [145]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [146]:
full_chain.invoke({"question": "how do I use Anthropic?"})

{'topic': '\nAnthropic', 'question': 'how do I use Anthropic?'}


' As Dario Amodei told me, an anthropic approach relies on anthropic reasoning, meaning that additional explanations or restrictions may be imposed on the way that a system is built or observed. It involves considering not just the laws of nature, but also the physical constraints imposed by our own existence within the universe. To use an anthropic approach, one must carefully consider the possible outcomes and implications of our own existence within a system, and using that information to make predictions and evaluations about the system as a whole.'

In [156]:
from langchain.schema import Document

In [163]:
all_docs = []
for idx, text_ in enumerate(["hello MASK", "hi", "there"]):
    all_docs.append(Document(page_content=text_, metadata={"id": f"doc_{idx+1}"}))

In [164]:
all_docs

[Document(page_content='hello MASK', metadata={'id': 'doc_1'}),
 Document(page_content='hi', metadata={'id': 'doc_2'}),
 Document(page_content='there', metadata={'id': 'doc_3'})]

In [274]:
def check_if_MASK_present(document: dict) -> bool:
    print (document)
    return "MASK" in document['page_content']


In [312]:
def combine_doc(input_data: dict):
    print (input_data)
    return Document(page_content=input_data['page_content'], metadata=input_data['metadata'])

In [313]:
prompt = PromptTemplate.from_template(
    """Remove the word MASK and return the modfied text from text below

Text: {page_content}
Answer:"""
)

coref_chain = RunnableParallel(
    {
    "page_content":  prompt | chat_gpt4 | StrOutputParser(),
    "metadata": itemgetter("metadata")
    }
) | RunnableLambda(combine_doc)

In [ ]:
coref_chain.invoke({"page_content": "hello", "metadata": "hello"})

In [315]:
coref_chain.get_graph().print_ascii()

            +--------------------------------------+                     
            | Parallel<page_content,metadata>Input |                     
            +--------------------------------------+                     
                   ****                  ****                            
               ****                          ****                        
             **                                  ****                    
+----------------+                                   **                  
| PromptTemplate |                                    *                  
+----------------+                                    *                  
          *                                           *                  
          *                                           *                  
          *                                           *                  
+-----------------+                                   *                  
| AzureChatOpenAI |                   

In [304]:
def route(info):
    print ("route data:", info)
    if info['is_MASK_present']:
        return coref_chain
    return combine_doc(info)

In [305]:
full_chain = {"is_MASK_present": RunnableLambda(check_if_MASK_present), "page_content": itemgetter("page_content"), "metadata": itemgetter("metadata")} | RunnableLambda(route)

In [306]:
all_docs = [
    {"page_content": "hello [MASK]", "metadata": {"doc_id": "id_123"}}
]

In [316]:
full_chain.invoke(all_docs[0])

{'page_content': 'hello [MASK]', 'metadata': {'doc_id': 'id_123'}}
route data: {'is_MASK_present': True, 'page_content': 'hello [MASK]', 'metadata': {'doc_id': 'id_123'}}


{'page_content': 'hello', 'metadata': {'doc_id': 'id_123'}}


Document(page_content='hello', metadata={'doc_id': 'id_123'})